In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, roc_auc_score

from shared.utils import load_data
from datasets import preprocess_dataset
from intrusion_detection_systems.models import cnn_model, mlp_model, rnn_model
from intrusion_detection_systems import train_dl_model, evaluate_dl_model
from agent.DQN_ML_Pool import DQNModelSelector
from agent.MAB_ML_Pool import MultiArmedBanditThompsonSampling

from art.estimators.classification.scikitlearn import ScikitlearnClassifier
from art.utils import to_categorical
from art.attacks.evasion import HopSkipJump
from art.estimators.classification import SklearnClassifier
from art.estimators.classification import PyTorchClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
name = "CIC-IDS_2017_2"
df = load_data(
            [
                "./shared/data/CIC_2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Friday-WorkingHours-Morning.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Monday-WorkingHours.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
                # "./shared/data/CIC_2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
                # "./shared/data/CIC_2017/Tuesday-WorkingHours.pcap_ISCX.csv"
            ],
            42
        )
print("Dataset loaded")
df_preprocessed = preprocess_dataset(
    df, save=True, dataset_type="CIC_2017", seed=42, load=False, name_save=name, name_load=name)
print("Dataset preprocessed")

X_train_full, y_train_full = df_preprocessed.x_train, df_preprocessed.y_train
X_test_full, y_test_full = df_preprocessed.x_test, df_preprocessed.y_test
print(f"Train full shape: {X_train_full.shape}, labels distribution: {np.unique(y_train_full, return_counts=True)}")
print(f"Test full shape: {X_test_full.shape}, labels distribution: {np.unique(y_test_full, return_counts=True)}")
y_train_full = np.array([int(str(x).strip()) for x in y_train_full])
y_test_full = np.array([int(str(x).strip()) for x in y_test_full])


Dataset loaded
Loading new data
labels: {'DDoS'}
Dataset preprocessed
Train full shape: (158021, 68), labels distribution: (array(['0', '1'], dtype=object), array([68402, 89619], dtype=int64))
Test full shape: (67724, 68), labels distribution: (array(['0', '1'], dtype=object), array([29316, 38408], dtype=int64))


In [8]:
# Train a NB Classifier
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train_full, y_train_full)
# Train a RF Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_full, y_train_full)
# Train a DT Classifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_full, y_train_full)
# Train a LR Classifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_full, y_train_full)

d:\anaconda3\envs\py39DQN\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [9]:
input_dim = X_train_full.shape[1]
#MLP Model
mlp_model_pt = mlp_model(input_dim)
criterion_mlp_model_pt = nn.CrossEntropyLoss()
optimizer_mlp_model_pt = optim.Adam(mlp_model_pt.parameters(), lr=0.001)

print("--- MLP Model ---")
print(mlp_model_pt)
print(f"Criterion: {criterion_mlp_model_pt}")
print(f"Optimizer: {optimizer_mlp_model_pt}")
# CNN Model
cnn_model_pt = cnn_model(input_dim)
criterion_cnn_model_pt = nn.CrossEntropyLoss()
optimizer_cnn_model_pt = optim.Adam(cnn_model_pt.parameters(), lr=0.001)

print("--- CNN Model ---")
print(cnn_model_pt)
print(f"Criterion: {criterion_cnn_model_pt}")
print(f"Optimizer: {optimizer_cnn_model_pt}")
# RNN Model
rnn_model_pt = rnn_model(input_dim) 
criterion_rnn_model_pt = nn.CrossEntropyLoss()
optimizer_rnn_model_pt = optim.Adam(rnn_model_pt.parameters(), lr=0.001)

print("--- RNN Model (LSTM) ---")
print(rnn_model_pt)
print(f"Criterion: {criterion_rnn_model_pt}")
print(f"Optimizer: {optimizer_rnn_model_pt}")

--- MLP Model ---
MLPModel(
  (fc1): Linear(in_features=68, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
Criterion: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
--- CNN Model ---
CNNModel(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(5,), stride=(1,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1792, out_features=2, bias=True)
)
Criterion: CrossEntropyLoss()
Optimizer: Adam

In [10]:
#Train DL models for attack
# Chia tập train -> train và validation
X_small = X_train_full[:1000]
y_small = y_train_full[:1000]
X_train, X_val, y_train, y_val = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42, stratify=y_small
)

# Chuyển sang tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_full[:1000], dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_full[:1000], dtype=torch.long)

# Dataloader
batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)


In [11]:
#MLP Target Model 
mlp_model_target = mlp_model(input_dim)
criterion_mlp_model_target = nn.CrossEntropyLoss()
optimizer_mlp_model_target = optim.Adam(mlp_model_target.parameters(), lr=0.001)

# CNN Target Model
cnn_model_target = cnn_model(input_dim)
criterion_cnn_model_target = nn.CrossEntropyLoss()
optimizer_cnn_model_target = optim.Adam(cnn_model_target.parameters(), lr=0.001)

# RNN Target Model
rnn_model_target = rnn_model(input_dim) 
criterion_rnn_model_target = nn.CrossEntropyLoss()
optimizer_rnn_model_target = optim.Adam(rnn_model_pt.parameters(), lr=0.001)

# ----- MLP -----
mlp_target = train_dl_model(mlp_model_target, train_loader, val_loader, criterion_mlp_model_target, optimizer_mlp_model_target, device, num_epochs=50)
evaluate_dl_model(mlp_target, test_loader, device,model_type='mlp', name="Test")

# ----- CNN -----
cnn_target = train_dl_model(cnn_model_target, train_loader, val_loader, criterion_cnn_model_target, optimizer_cnn_model_target, device,num_epochs=50)
evaluate_dl_model(cnn_target, test_loader, device,model_type='cnn', name="Test")

# ----- RNN -----
rnn_target = train_dl_model(rnn_model_target, train_loader, val_loader, criterion_rnn_model_target, optimizer_rnn_model_target, device,num_epochs=50)
evaluate_dl_model(rnn_target, test_loader, device,model_type='rnn', name="Test")


--- Training MLPModel ---
Epoch [1/50], Train Loss: 0.6900, Train Acc: 0.5650, Val Loss: 0.6784, Val Acc: 0.6550
Epoch [2/50], Train Loss: 0.6734, Train Acc: 0.6000, Val Loss: 0.6606, Val Acc: 0.5750
Epoch [3/50], Train Loss: 0.6558, Train Acc: 0.6012, Val Loss: 0.6393, Val Acc: 0.6550
Epoch [4/50], Train Loss: 0.6361, Train Acc: 0.6500, Val Loss: 0.6120, Val Acc: 0.7200
Epoch [5/50], Train Loss: 0.6091, Train Acc: 0.7025, Val Loss: 0.5778, Val Acc: 0.7400
Epoch [6/50], Train Loss: 0.5783, Train Acc: 0.7188, Val Loss: 0.5395, Val Acc: 0.7700
Epoch [7/50], Train Loss: 0.5413, Train Acc: 0.7312, Val Loss: 0.4954, Val Acc: 0.7700
Epoch [8/50], Train Loss: 0.4983, Train Acc: 0.7475, Val Loss: 0.4458, Val Acc: 0.7850
Epoch [9/50], Train Loss: 0.4504, Train Acc: 0.7650, Val Loss: 0.3923, Val Acc: 0.8250
Epoch [10/50], Train Loss: 0.3982, Train Acc: 0.7875, Val Loss: 0.3356, Val Acc: 0.8550
Epoch [11/50], Train Loss: 0.3430, Train Acc: 0.8812, Val Loss: 0.2768, Val Acc: 0.9450
Epoch [12/50],

In [12]:
SUB_TRAIN = min(1000, len(X_train_full))
SUB_TEST = min(2000, len(X_test_full))
# Sử dụng mẫu nhỏ hoặc toàn bộ dữ liệu
X_train, y_train = X_train_full[:SUB_TRAIN], y_train_full[:SUB_TRAIN]
X_test, y_test = X_test_full[:SUB_TEST], y_test_full[:SUB_TEST]
# Nếu muốn dùng toàn bộ dữ liệu, bỏ dòng trên và dùng:
# X_train, y_train = X_train_full, y_train_full
# X_test, y_test = X_test_full, y_test_full
print("📌 Nhãn của tập huấn luyện (y_train):", np.unique(y_train, return_counts=True))
print("📌 Nhãn của tập kiểm thử (y_test):", np.unique(y_test, return_counts=True))

📌 Nhãn của tập huấn luyện (y_train): (array([0, 1]), array([423, 577], dtype=int64))
📌 Nhãn của tập kiểm thử (y_test): (array([0, 1]), array([ 855, 1145], dtype=int64))


In [13]:
# =====================
# DQN Training and Evaluation
# =====================
models = [
    RandomForestClassifier(random_state=42),
    DecisionTreeClassifier(random_state=42),
    GaussianNB()
]

dqn = DQNModelSelector(models, n_clusters=3)

print("Training DQN-based selector with cluster-specific DQNs...")
dqn.train(X_train, y_train)

print("Predicting on test set...")
y_pred_dqn, arms = dqn.predict(X_test)

if isinstance(y_test, pd.DataFrame):
    if "Label" in y_test.columns:
        y_test = y_test["Label"].values
    else:
        y_test = y_test.iloc[:, 0].values
elif isinstance(y_test, pd.Series):
    y_test = y_test.values

y_test = np.array([int(str(x).strip()) for x in y_test])
y_pred_dqn = np.array([int(x) for x in y_pred_dqn])

print(classification_report(y_test, y_pred_dqn))
print("Accuracy:", accuracy_score(y_test, y_pred_dqn))
print("Recall (macro):", recall_score(y_test, y_pred_dqn, average='macro'))
print("F1 Score (macro):", f1_score(y_test, y_pred_dqn, average='macro'))
print("ROC AUC:", roc_auc_score(y_test, y_pred_dqn))

Training DQN-based selector with cluster-specific DQNs...

 Training models for cluster 0


d:\anaconda3\envs\py39DQN\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0, Epoch 1/30, Avg Reward: 0.9908
Cluster 0, Epoch 2/30, Avg Reward: 0.9922
Cluster 0, Epoch 3/30, Avg Reward: 0.9905
Cluster 0, Epoch 4/30, Avg Reward: 0.9988
Cluster 0, Epoch 5/30, Avg Reward: 0.9946
Cluster 0, Epoch 6/30, Avg Reward: 1.0039
Cluster 0, Epoch 7/30, Avg Reward: 0.9990
Cluster 0, Epoch 8/30, Avg Reward: 0.9968
Cluster 0, Epoch 9/30, Avg Reward: 1.0003
Cluster 0, Epoch 10/30, Avg Reward: 0.9982
Cluster 0, Epoch 11/30, Avg Reward: 1.0069
Cluster 0, Epoch 12/30, Avg Reward: 1.0057
Cluster 0, Epoch 13/30, Avg Reward: 0.9780
Cluster 0, Epoch 14/30, Avg Reward: 0.9990
Cluster 0, Epoch 15/30, Avg Reward: 1.0009
Cluster 0, Epoch 16/30, Avg Reward: 1.0019
Cluster 0, Epoch 17/30, Avg Reward: 1.0012
Cluster 0, Epoch 18/30, Avg Reward: 1.0138
Cluster 0, Epoch 19/30, Avg Reward: 1.0005
Cluster 0, Epoch 20/30, Avg Reward: 0.9971
Cluster 0, Epoch 21/30, Avg Reward: 1.0039
Cluster 0, Epoch 22/30, Avg Reward: 0.9982
Cluster 0, Epoch 23/30, Avg Reward: 1.0036
Cluster 0, Epoch 24/

In [14]:
# Train the MAB
mab = MultiArmedBanditThompsonSampling(n_arms=3, n_clusters=2)
mab.train(X_train, y_train)

d:\anaconda3\envs\py39DQN\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0: 96 samples
Training arm 0 on cluster 0
cluster_X_train shape: (96, 68)
cluster_y_train shape: (96,)
arm_mask shape: (96,)
Training arm 1 on cluster 0
cluster_X_train shape: (96, 68)
cluster_y_train shape: (96,)
arm_mask shape: (96,)
Training arm 2 on cluster 0
cluster_X_train shape: (96, 68)
cluster_y_train shape: (96,)
arm_mask shape: (96,)
Cluster 1: 904 samples
Training arm 0 on cluster 1
cluster_X_train shape: (904, 68)
cluster_y_train shape: (904,)
arm_mask shape: (904,)
Training arm 1 on cluster 1
cluster_X_train shape: (904, 68)
cluster_y_train shape: (904,)
arm_mask shape: (904,)
Training arm 2 on cluster 1
cluster_X_train shape: (904, 68)
cluster_y_train shape: (904,)
arm_mask shape: (904,)
Setting reward_sums arm 0 on cluster 0
Setting reward_sums arm 1 on cluster 0
Setting reward_sums arm 2 on cluster 0
Setting reward_sums arm 0 on cluster 1
Setting reward_sums arm 1 on cluster 1
Setting reward_sums arm 2 on cluster 1


In [15]:
y_pred_mab, selected_arms = mab.predict(X_test)

if isinstance(y_test, pd.DataFrame):
    if "Label" in y_test.columns:
        y_test = y_test["Label"].values
    else:
        y_test = y_test.iloc[:, 0].values
elif isinstance(y_test, pd.Series):
    y_test = y_test.values

y_test = np.array([int(str(x).strip()) for x in y_test])
y_pred_mab = np.array([int(x) for x in y_pred_mab])

print(classification_report(y_test, y_pred_mab))
print("Accuracy:", accuracy_score(y_test, y_pred_mab))
print("Recall (macro):", recall_score(y_test, y_pred_mab, average='macro'))
print("F1 Score (macro):", f1_score(y_test, y_pred_mab, average='macro'))
print("ROC AUC:", roc_auc_score(y_test, y_pred_mab))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       855
           1       1.00      0.99      1.00      1145

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

Accuracy: 0.9945
Recall (macro): 0.9947521642534283
F1 Score (macro): 0.994386000727268
ROC AUC: 0.9947521642534283


In [16]:
attacks_data = X_test_full[y_test_full == 1]
normal_data = X_test_full[y_test_full == 0]
print(attacks_data.shape, normal_data.shape)

FUNCTIONAL_FEATURES = [
 ' min_seg_size_forward',' Bwd Header Length',' Destination Port'
 'Init_Win_bytes_forward',' Init_Win_bytes_backward',' Bwd Packets/s'
 'Total Length of Fwd Packets',' Subflow Fwd Bytes',' Max Packet Length'
 'Bwd Packet Length Max',' Avg Bwd Segment Size',' Bwd Packet Length Mean'
 ' Fwd Packet Length Max',' Average Packet Size',' Packet Length Std'
 ' Packet Length Mean',' Bwd Packet Length Std',' Bwd Packet Length Min'
 ' Fwd Packet Length Std',' Fwd Packet Length Min',' Min Packet Length'
 ' Fwd Packet Length Mean',' Avg Fwd Segment Size',' act_data_pkt_fwd'
 ' Total Fwd Packets','Subflow Fwd Packets',' Total Backward Packets']
print(len(FUNCTIONAL_FEATURES))
FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c not in FUNCTIONAL_FEATURES][:-1]
print(FUNCTIONAL_FEATURES_IDEXES, len(FUNCTIONAL_FEATURES_IDEXES))
NON_FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c in FUNCTIONAL_FEATURES]
print(NON_FUNCTIONAL_FEATURES_IDEXES, len(NON_FUNCTIONAL_FEATURES_IDEXES))
print(len(df.columns))

(38408, 68) (29316, 68)
19
[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 51, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66] 56
[3, 7, 13, 32, 48, 49, 50, 52, 53, 57, 59] 11
68


In [17]:
# Create ART classifier
class Classifier(ScikitlearnClassifier):
    
    def __init__(self, model, clip_values=None, preprocessing=(0, 1), attacks=[]):
        super(Classifier, self).__init__(model=model, clip_values=clip_values, preprocessing=preprocessing)
        self._attacks = attacks

    def predict(self, x, **kwargs):
        # Set attacks features to X
        for i in FUNCTIONAL_FEATURES_IDEXES:
            for j in range(len(x)):
                x[j][i] = self._attacks[j][i]
        predictions = self._model.predict(x)
        return to_categorical(predictions, nb_classes=self._get_nb_classes())

In [ ]:
models = [nb, dt, lr, rf]
accuracys, drs, f1s, rocs = [], [], [], []
mab_accuracys, mab_drs, mab_f1s, mab_rocs = [], [], [], []

for model in models:
    classifier = SklearnClassifier(model=model, clip_values=(0, 1))
    attack = HopSkipJump(classifier=classifier, targeted=False, max_iter=10,  max_eval=1000, init_eval=10, init_size=20)
    
    x_test_adv = attack.generate(attacks_data[:500], np.zeros((attacks_data[:500].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:500], normal_data[:500]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:500].shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:500]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:500].shape[0], 1))))

    print("====================> Model: ", model)
    print("---------- Adversarial data")

    # Predict cho dqn
    y_true = adv_y_test.astype(int).ravel()
    y_pred_dqn = dqn.predict(adv_x_test)[0].astype(int).ravel()

    accuracy = accuracy_score(y_true, y_pred_dqn)
    recall = recall_score(y_true, y_pred_dqn)
    f1 = f1_score(y_true, y_pred_dqn)
    roc = roc_auc_score(y_true, y_pred_dqn)

    accuracys.append(accuracy)
    drs.append(recall)
    f1s.append(f1)
    rocs.append(roc)

    print("===> DQN:")
    print(classification_report(y_true, y_pred_dqn))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

    # Predict cho mab
    y_pred_mab = mab.predict(adv_x_test)[0].astype(int).ravel()

    mab_accuracy = accuracy_score(y_true, y_pred_mab)
    mab_recall = recall_score(y_true, y_pred_mab)
    mab_f1 = f1_score(y_true, y_pred_mab)
    mab_roc = roc_auc_score(y_true, y_pred_mab)

    mab_accuracys.append(mab_accuracy)
    mab_drs.append(mab_recall)
    mab_f1s.append(mab_f1)
    mab_rocs.append(mab_roc)

    print("===> MAB:")
    print(classification_report(y_true, y_pred_mab))
    print("Accuracy: ", mab_accuracy)
    print("Detection Rate: ", mab_recall)
    print("F1 Score: ", mab_f1)
    print("ROC AUC Score: ", mab_roc)

# Tổng kết kết quả
print("=====================================")
print("=== DQN ===")
print("Accuracy: ", sum(accuracys) / len(accuracys))
print("Detection Rate: ", sum(drs) / len(drs))
print("F1 Score: ", sum(f1s) / len(f1s))
print("ROC AUC Score: ", sum(rocs) / len(rocs))

print("=== MAB ===")
print("Accuracy: ", sum(mab_accuracys) / len(mab_accuracys))
print("Detection Rate: ", sum(mab_drs) / len(mab_drs))
print("F1 Score: ", sum(mab_f1s) / len(mab_f1s))
print("ROC AUC Score: ", sum(mab_rocs) / len(mab_rocs))

HopSkipJump:   0%|          | 0/500 [00:00<?, ?it/s]

HopSkipJump: 100%|██████████| 500/500 [00:12<00:00, 38.47it/s]


====================> Model:  GaussianNB()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 330 lần
  → Mô hình 1: 347 lần
  → Mô hình 2: 323 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.997
Detection Rate:  0.998
F1 Score:  0.997002997002997
ROC AUC Score:  0.997
===> MAB:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       500
           1       1.00      0.97      0.98       500

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000

Accuracy:  0.982
Detection Rate:  0.966
F1 Sc

HopSkipJump: 100%|██████████| 500/500 [00:08<00:00, 57.20it/s]


====================> Model:  DecisionTreeClassifier()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 379 lần
  → Mô hình 1: 345 lần
  → Mô hình 2: 276 lần
===> DQN:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       500
           1       1.00      0.99      0.99       500

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

Accuracy:  0.993
Detection Rate:  0.99
F1 Score:  0.9929789368104314
ROC AUC Score:  0.993
===> MAB:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       500
           1       1.00      0.98      0.99       500

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000

Accuracy:  0.992
Detection Rate: 

HopSkipJump: 100%|██████████| 500/500 [00:09<00:00, 53.76it/s]


====================> Model:  LogisticRegression()
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 395 lần
  → Mô hình 1: 266 lần
  → Mô hình 2: 339 lần
===> DQN:
              precision    recall  f1-score   support

           0       0.70      1.00      0.82       500
           1       0.99      0.57      0.73       500

    accuracy                           0.78      1000
   macro avg       0.85      0.78      0.77      1000
weighted avg       0.85      0.78      0.77      1000

Accuracy:  0.784
Detection Rate:  0.572
F1 Score:  0.7258883248730965
ROC AUC Score:  0.784
===> MAB:
              precision    recall  f1-score   support

           0       0.55      1.00      0.71       500
           1       0.99      0.17      0.29       500

    accuracy                           0.58      1000
   macro avg       0.77      0.58      0.50      1000
weighted avg       0.77      0.58      0.50      1000

Accuracy:  0.583
Detection Rate:  0.

In [23]:
# Evaluate with multiple models
models = [mlp_target, cnn_target, rnn_target]
criterions = [criterion_mlp_model_target, criterion_cnn_model_target, criterion_rnn_model_target]
optimizers = [optimizer_mlp_model_target, optimizer_cnn_model_target, optimizer_rnn_model_target]
input_shapes = [(input_dim,), (input_dim, 1), (input_dim, 1)]
model_types = ['mlp', 'cnn', 'rnn']
accuracys, drs, f1s, rocs = [], [], [], []
mab_accuracys, mab_drs, mab_f1s, mab_rocs = [], [], [], []
attacks_data = attacks_data.astype(np.float32)
normal_data = normal_data.astype(np.float32)
for i, model in enumerate(models):
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0.0, 1.0),
        loss=criterions[i],
        optimizer=optimizers[i],
        input_shape=input_shapes[i],
        nb_classes=2,
    )

    print(f"\n======> Attacking model: {model_types[i].upper()}")
    attack = HopSkipJump(classifier=classifier, targeted=False, max_iter=5,  max_eval=500, init_eval=10, init_size=20)
    # Reshape dữ liệu theo loại model
    if model_types[i] == 'cnn':
        x_input = attacks_data[:500].reshape((500, input_dim, 1))
        normal_data_reshaped = normal_data[:500].reshape((500, input_dim, 1))
    elif model_types[i] == 'rnn':
        x_input = attacks_data[:500].reshape((500, input_dim, 1))
        normal_data_reshaped = normal_data[:500].reshape((500, input_dim, 1))
    else:
        x_input = attacks_data[:500]
        normal_data_reshaped = normal_data[:500]

    # Sinh mẫu tấn công
    attack = HopSkipJump(classifier=classifier, targeted=False, max_iter=10, max_eval=1000, init_eval=10, init_size=20)
    x_test_adv = attack.generate(x_input, np.zeros((x_input.shape[0], 1)))

    # Gộp với dữ liệu bình thường
    adv_x_test = np.concatenate((x_test_adv, normal_data_reshaped))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data_reshaped.shape[0], 1))))
    adv_x_test = adv_x_test.astype(np.float64)
    y_true = adv_y_test.astype(int).ravel()
    if adv_x_test.ndim == 3:
        adv_x_test = adv_x_test.reshape((adv_x_test.shape[0], -1))
    print("====================> Model: ", model)
    print("---------- Adversarial data")

    # Predict cho dqn
    y_pred_dqn  = dqn.predict(adv_x_test)[0].astype(int).ravel()

    accuracy = accuracy_score(y_true, y_pred_dqn )
    recall = recall_score(y_true, y_pred_dqn )
    f1 = f1_score(y_true, y_pred_dqn )
    roc = roc_auc_score(y_true, y_pred_dqn )

    accuracys.append(accuracy)
    drs.append(recall)
    f1s.append(f1)
    rocs.append(roc)

    print("===> DQN:")
    print(classification_report(y_true, y_pred_dqn))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

    # Predict cho mab
    y_pred_mab = mab.predict(adv_x_test)[0].astype(int).ravel()

    mab_accuracy = accuracy_score(y_true, y_pred_mab)
    mab_recall = recall_score(y_true, y_pred_mab)
    mab_f1 = f1_score(y_true, y_pred_mab)
    mab_roc = roc_auc_score(y_true, y_pred_mab)

    mab_accuracys.append(mab_accuracy)
    mab_drs.append(mab_recall)
    mab_f1s.append(mab_f1)
    mab_rocs.append(mab_roc)

    print("===> MAB:")
    print(classification_report(y_true, y_pred_mab))
    print("Accuracy: ", mab_accuracy)
    print("Detection Rate: ", mab_recall)
    print("F1 Score: ", mab_f1)
    print("ROC AUC Score: ", mab_roc)

# Tổng kết kết quả
print("=====================================")
print("=== DQN ===")
print("Accuracy: ", sum(accuracys) / len(accuracys))
print("Detection Rate: ", sum(drs) / len(drs))
print("F1 Score: ", sum(f1s) / len(f1s))
print("ROC AUC Score: ", sum(rocs) / len(rocs))

print("=== MAB ===")
print("Accuracy: ", sum(mab_accuracys) / len(mab_accuracys))
print("Detection Rate: ", sum(mab_drs) / len(mab_drs))
print("F1 Score: ", sum(mab_f1s) / len(mab_f1s))
print("ROC AUC Score: ", sum(mab_rocs) / len(mab_rocs))


======> Attacking model: MLP


HopSkipJump: 100%|██████████| 500/500 [00:06<00:00, 79.37it/s]


====================> Model:  MLPModel(
  (fc1): Linear(in_features=68, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 341 lần
  → Mô hình 1: 496 lần
  → Mô hình 2: 163 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       500
           1       0.99      0.99      0.99      

HopSkipJump: 100%|██████████| 500/500 [01:19<00:00,  6.27it/s]


====================> Model:  CNNModel(
  (conv1): Conv1d(1, 64, kernel_size=(5,), stride=(1,))
  (relu1): ReLU()
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(5,), stride=(1,))
  (relu2): ReLU()
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=1792, out_features=2, bias=True)
)
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 402 lần
  → Mô hình 1: 256 lần
  → Mô hình 2: 342 lần
===> DQN:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86       500
           1       0.99      0.67      0.80       500

    accuracy                           0.83      1000
   macro avg       0.87      0.83      0.83      1000
weighted avg       0.87      0.83      0.83      1000

Accuracy:  0.834
Detection Rate:  0.672
F1 Score:  0.8019093078758951
ROC AUC Score:  0.8340

HopSkipJump: 100%|██████████| 500/500 [00:13<00:00, 36.74it/s]


====================> Model:  RNNModel(
  (lstm): LSTM(1, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)
---------- Adversarial data

[🔍] Thống kê số lần mỗi ARM (mô hình) được chọn:
  → Mô hình 0: 341 lần
  → Mô hình 1: 496 lần
  → Mô hình 2: 163 lần
===> DQN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       500
           1       1.00      1.00      1.00       500

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy:  0.998
Detection Rate:  1.0
F1 Score:  0.998003992015968
ROC AUC Score:  0.998
===> MAB:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       500
           1       1.00      0.99      0.99       500

    accuracy                       